# 读取2月的FNL文件风场，与模拟结果、观测结果比较

* Author: Evan
* date: 2023-04-26

In [1]:
import xarray as xr
import cfgrib
import numpy as np
import pandas as pd

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [2]:
path = 'F:/Data/case_anqing/FNL4wind/'
dates = np.arange(1,29)
hours = np.arange(0,20,6)

dslist = []
for date in dates:
    for hour in hours:
        file = path + 'fnl_202302' + str(date).zfill(2) + '_' + str(hour).zfill(2) + '_00.grib2'
        ds = xr.open_dataset(file,
                     engine='cfgrib',
                     backend_kwargs={
                         'filter_by_keys':{
                             'typeOfLevel':'isobaricInhPa',
                             },
                         'errors':'ignore',})
        dslist.append(ds)

dsout = xr.concat(dslist,dim='time')


In [3]:
fnlout = dsout.assign_coords(time = dsout.time + pd.Timedelta(hours=8))
fnlout

<xarray.Dataset>
Dimensions:        (time: 112, isobaricInhPa: 33, latitude: 181, longitude: 360)
Coordinates:
  * time           (time) datetime64[ns] 2023-02-01T08:00:00 ... 2023-03-01T0...
    step           timedelta64[ns] 00:00:00
  * isobaricInhPa  (isobaricInhPa) float64 1e+03 975.0 950.0 ... 3.0 2.0 1.0
  * latitude       (latitude) float64 90.0 89.0 88.0 87.0 ... -88.0 -89.0 -90.0
  * longitude      (longitude) float64 0.0 1.0 2.0 3.0 ... 357.0 358.0 359.0
    valid_time     (time) datetime64[ns] 2023-02-01 ... 2023-02-28T18:00:00
Data variables:
    gh             (time, isobaricInhPa, latitude, longitude) float32 213.6 ....
    t              (time, isobaricInhPa, latitude, longitude) float32 247.1 ....
    r              (time, isobaricInhPa, latitude, longitude) float32 100.0 ....
    q              (time, isobaricInhPa, latitude, longitude) float32 0.00035...
    w              (time, isobaricInhPa, latitude, longitude) float32 0.00819...
    wz             (time, isobaricInhPa, latitude, longitude) float32 -0.0004...
    u              (time, isobaricInhPa, latitude, longitude) float32 -6.237 ...
    v              (time, isobaricInhPa, latitude, longitude) float32 -0.8215...
    absv           (time, isobaricInhPa, latitude, longitude) float32 0.00013...
    o3mr           (time, isobaricInhPa, latitude, longitude) float32 5.291e-...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2023-04-26T10:22 GRIB to CDM+CF via cfgrib-0.9.1...

In [4]:
compression=dict(zlib=True,complevel=5)
encoding={var:compression for var in fnlout.data_vars}
fnlout.to_netcdf('D:/Download/fnl_Feb.nc',encoding=encoding)